# Import Required Packages

In [1]:
# Imports
import os
import datetime
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Input data from User

In [2]:
#Market analysed: 'Investment','FullYear','DayAhead','Balancing' (choose one or several)
markets=['FullYear'] 
output='BalanceHeatHourly'
first_timestep="2012-01-02"
#Meaning of SSS and TTT in the data: 'DaysHours','Hours5min','WeeksHours'
meaning_SSS_TTT='DaysHours'
#Time size of each time step in TTT for creating timestamp
size_timestep="3600s"


# Plot Settings

In [3]:
# Set plotting specifications
%matplotlib inline
plt.rcParams.update({'font.size': 21})
plt.rcParams['xtick.major.pad']='12'
plt.rc('legend', fontsize=16)
y_limit = 1.1
lw = 3

# Read Input Files

In [4]:
data=pd.DataFrame()
for market in markets:
    csvfiles = []
    for file in glob.glob("./input/results/" + market + "/*.csv"):
        csvfiles.append(file)

    csvfiles=[file.replace('./input\\','') for file in csvfiles] 
    csvfiles=[file.replace('.csv','') for file in csvfiles]  
    csvfiles=[file.split('_') for file in csvfiles]  
    csvfiles = np.asarray(csvfiles)  
    csvfiles=pd.DataFrame.from_records(csvfiles)
    
    csvfiles.rename(columns={0: 'Output', 1: 'Scenario',2: 'Year',3:'Subset'}, inplace=True)
    scenarios=csvfiles.Scenario.unique().tolist()
    years=csvfiles.Year.unique().tolist()
    subsets=csvfiles.Subset.unique().tolist()

    for scenario in scenarios:
        for year in years:
            for subset in subsets:
                file = "./input/results/"+ market + "/"+ output + "_" + scenario + "_" + year + "_" + subset + ".csv"
                if os.path.isfile(file):
                    df=pd.read_csv(file,encoding='utf8')
                    df['Scenario'] = scenario
                    df['Market']   = market
                    #Renaming columns just in case timeconversion was required
                    df.rename(columns = {'G':'GGG', 'C':'CCC', 'Y':'YYY','TTT_NEW':'TTT','SSS_NEW':'SSS'}, inplace = True) 
                    data=data.append(df) 

                
                

C:\Users\s151529\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning:

Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.



In [5]:
#Timestamp addition
full_timesteps = pd.read_csv('./input/full_timesteps_'+meaning_SSS_TTT+'.csv')
full_timesteps.Key=full_timesteps['SSS']+full_timesteps['TTT']
number_periods=len(full_timesteps.Key.unique())
full_timesteps['timestamp']= pd.date_range(first_timestep, periods = number_periods, freq =size_timestep)
dict_timestamp=dict(zip(full_timesteps.Key, full_timesteps.timestamp))
data['timestamp']=data['SSS']+data['TTT']
data['timestamp']=data['timestamp'].map(dict_timestamp)



C:\Users\s151529\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



In [6]:
#Additional set declaration
ccc = list(data.CCC.unique())
rrr = list(data.RRR.unique())
aaa = list(data.AAA.unique())
heatbaltype = list(data.H_BAL_TYPE.unique())
yyy = list(data.YYY.unique())
sss = list(full_timesteps.SSS.unique())
ttt = list(full_timesteps.TTT.unique())


In [42]:
heatbaltype

['CHP-BACK-PRESSURE',
 'BOILERS',
 'NETEXPORT',
 'PRICE',
 'DEMAND_EXO',
 'DEMAND_INTERSTO',
 'INTER-STO',
 'P2H',
 'DEMAND_INTRASTO',
 'INTRA-STO',
 'CHP-EXTRACTION',
 'SOLAR-HEATING',
 'CURTAILMENT',
 'DEMAND_P2G']

In [8]:
#Replace possible "Eps" with 0
data.Val=data.Val.replace('Eps', 0)
data.Val=pd.to_numeric(data.Val)

# Make output directories


In [9]:
# Make output folder
if not os.path.isdir('output'):
    os.makedirs('output')

In [10]:
# Make output folder
if not os.path.isdir('output/' + output):
    os.makedirs('output/' + output)

# Plot filter selection (user selection)

In [11]:
# Scenario to investigate
#All scenarios
scenario = scenarios
# Or select scenarios by names 
#scenario_names = ['NoSectorCoupling']
#scenario = scenario_names

In [12]:
# Market to investigate
#All markets
market = markets
# Or select market by names 
#market_names = ['DayAhead']
#market = market_names

In [13]:
# Areas to investigate
#All areas
a = aaa
# Or select areas by names 
#area_names = ['FR_IND-HT-NODH']
#a = area_names

In [14]:
# Years to investigate
#All years
y = yyy
# Or select yyy by names 
#year_names = ['2045']
#y = year_names

In [15]:
# Seasons to investigate
#All seasons
s = sss
# Or select seasons by names 
#season_names = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007']
#s = season_names

In [16]:
# Terms to investigate
#All terms
t = ttt
# Or select terms by names 
# term_names = ['T001', 'S007', 'S020', 'S024', 'S028', 'S038', 'S042', 'S043']
# t = term_names

In [17]:
data_plot=data[(data.SSS.isin(s)) & (data.TTT.isin(t)) & (data.YYY.isin(y)) & (data.AAA.isin(a)) 
               & (data.Scenario.isin(scenario)) & (data.Market.isin(market))]

# Plotting

In [30]:
table = pd.pivot_table(data_plot, values='Val', index=['Scenario','YYY','Market','timestamp'],
...                     columns=['H_BAL_TYPE'], aggfunc=np.sum, fill_value=0).reset_index()

In [31]:
table_names=list(table.columns.values)

In [32]:
table.timestamp=table.timestamp.astype('datetime64[ns]')
table.sort_values(by='timestamp',inplace=True)

In [33]:
##Combining columns (user specification)
#CHP
if 'CHP-EXTRACTION' not in table_names:
    table['CHP-EXTRACTION']=0 
if 'CHP-BACK-PRESSURE' not in table_names:
    table['CHP-BACK-PRESSURE']=0 
table['CHP']=table['CHP-BACK-PRESSURE']+table['CHP-EXTRACTION']
del table['CHP-BACK-PRESSURE']
del table['CHP-EXTRACTION'] 


#STORAGE PRODUCTION
if 'INTRA-STO' not in table_names:
    table['INTRA-STO']=0 
if 'INTER-STO' not in table_names:
    table['INTER-STO']=0 
table['H-STO']=table['INTRA-STO']+table['INTER-STO']
del table['INTER-STO']
del table['INTRA-STO']

#STORAGE DEMAND
if 'DEMAND_INTERSTO' not in table_names:
    table['DEMAND_INTERSTO']=0 
if 'DEMAND_INTRASTO' not in table_names:
    table['DEMAND_INTRASTO']=0 
table['DEMAND_STO']=table['DEMAND_INTERSTO']+table['DEMAND_INTRASTO']
del table['DEMAND_INTERSTO']
del table['DEMAND_INTRASTO']

table_names=list(table.columns.values)

In [34]:
all_pro=['BOILERS','CHP-BACK-PRESSURE', 'CHP-EXTRACTION', 'INTRA-STO', 'INTER-STO','P2H','H-STO','CHP','SOLAR-HEATING','FUELCELL']
prod_list=list(set(table_names).intersection(all_pro))
all_dem=['DEMAND_EXO','DEMAND_INTERSTO','DEMAND_INTRASTO','DEMAND_STO','DEMAND_P2G','DEMAND_DISTLOSSES']
dem_list=list(set(table_names).intersection(all_dem))

In [35]:
x=np.array(table.timestamp)

In [36]:
title_subplots=[]
name_subplots=[]
for mark in market:
    for sce in scenarios:
        for year in y:
            title_subplots.append(mark + ' ' + sce + ' ' + str(year))
            name_subplots.append(mark + sce + str(year))

In [37]:
plot_spec=[]
for mark in market:
    for sce in scenarios:
        for year in y:
            plot_spec.append([dict({"secondary_y": True})])

##  Heat balance

In [1]:
fig = make_subplots(rows=len(scenario)*len(y)*len(market), cols=1,subplot_titles=title_subplots,specs=plot_spec, vertical_spacing = 0.05)
for mark in market:
    for sce in scenario:
        for year in y:
            #Production
            for i in prod_list:
                fig.add_trace(go.Scatter(x=table.loc[(table['Scenario'] == sce) & (table['Market'] == mark) & (table['YYY'] == year)].timestamp, y=table.loc[(table['Scenario'] == sce) & (table['Market'] == mark) & (table['YYY'] == year)][i], stackgroup='one', fill='tozeroy',name=i),
                             row=name_subplots.index(mark+sce+str(year))+1, col=1)
            for i in dem_list:
            #Demand 
                fig.add_trace(go.Scatter(x=table.loc[(table['Scenario'] == sce) & (table['Market'] == mark) & (table['YYY'] == year)].timestamp, y=table.loc[(table['Scenario'] == sce) & (table['Market'] == mark) & (table['YYY'] == year)][i],name=i), 
                              row=name_subplots.index(mark+sce+str(year))+1, col=1) 
            #Price
            fig.add_trace(
            go.Scatter(x=table.loc[(table['Scenario'] == sce) & (table['Market'] == mark) & (table['YYY'] == year)].timestamp, y=table.loc[(table['Scenario'] == sce) & (table['Market'] == mark) & (table['YYY'] == year)]['PRICE'], name="Price"),
            secondary_y=True,row=name_subplots.index(mark+sce+str(year))+1, col=1)  
fig.update_layout(title="Heat balance") 
fig.update_yaxes(title_text="Heat (MWh/h)", secondary_y=False)
fig.update_yaxes(title_text="Price (€/MWh)", secondary_y=True)
fig.update_layout(legend_orientation="h",width=1000,height=1500)
fig.show()

NameError: name 'make_subplots' is not defined

## Heat production

In [2]:
fig = make_subplots(rows=len(scenario)*len(y)*len(market), cols=1,subplot_titles=title_subplots, vertical_spacing = 0.05)
for mark in market:
    for sce in scenario:
        for year in y:
            #Production
            for i in prod_list:
                fig.add_trace(go.Scatter(x=table.loc[(table['Scenario'] == sce) & (table['Market'] == mark) & (table['YYY'] == year)].timestamp, y=table.loc[(table['Scenario'] == sce) & (table['Market'] == mark) & (table['YYY'] == year)][i],name=i),
                             row=name_subplots.index(mark+sce+str(year))+1, col=1)
fig.update_layout(
    title="Heat production",
)  
fig.update_yaxes(title_text="Energy (MWh/h)", secondary_y=False)
fig.update_layout(legend_orientation="h",width=1000,height=1500)
fig.show()


NameError: name 'make_subplots' is not defined

## Heat demand

In [3]:
fig = make_subplots(rows=len(scenario)*len(y)*len(market), cols=1,subplot_titles=title_subplots, vertical_spacing = 0.05)
#Demand df
for mark in market:
    for sce in scenario:
        for year in y:
            for i in dem_list:
                fig.add_trace(go.Scatter(x=table.loc[(table['Scenario'] == sce) & (table['Market'] == mark) & (table['YYY'] == year)].timestamp, y=table.loc[(table['Scenario'] == sce) & (table['Market'] == mark) & (table['YYY'] == year)][i],name=i), 
                              row=name_subplots.index(mark+sce+str(year))+1, col=1) # fill down to xaxis
                fig.update_layout(
                    title="Heat demand",
                )
fig.update_yaxes(title_text="Energy (MWh/h)", secondary_y=False)
fig.update_layout(legend_orientation="h",width=1000,height=1500)
fig.show()

NameError: name 'make_subplots' is not defined

In [41]:
#Not dynamic plot

#pal=sns.color_palette("Set2")
#plt.figure(figsize=(21,7))
#plt.stackplot(x,prod, labels=pro_label,colors=pal)
#plt.legend(loc='lower left',bbox_to_anchor=(0.05, -0.35), fancybox=True, shadow=True, ncol=6,fontsize=12)
#plt.title('Energy balance',fontsize=16)
#plt.ylabel('Energy (MWh/h)',fontsize=16)
#plt.xticks(rotation=90,fontsize=14)
#plt.yticks(fontsize=14)
#for i in dem_list:
#    plt.plot(x,table[i])
#plt.legend(loc='lower left',bbox_to_anchor=(0.05, -0.35),fancybox=True, shadow=True, ncol=6,fontsize=12)
#plt.twinx()
#plt.plot(x,table['PRICE'], ':k')
#plt.ylabel('Price (€/MWh)',fontsize=16)
#plt.yticks(fontsize=14)
#plt.legend(loc='lower right',bbox_to_anchor=(0.95, -0.35),fancybox=True, shadow=True, fontsize=12)
#plt.margins(x=0)
#plt.savefig('Balance.png', bbox_inches='tight', dpi=1000)
#plt.show()